In [40]:
import pandas as pd
import numpy as np
import re

PATH = '/Users/danielsagher/Dropbox/Documents/projects/bls_api_project'

national_results = pd.read_csv('../api_call/main_output/national_results.csv')
state_results = pd.read_csv('../api_call/main_output/state_results.csv')
national_series_dimension = pd.read_csv('../csv_from_excel/national_series_dimension_w_adjust.csv')
state_series_dimension = pd.read_csv('../state_scrape/output/state_series_dimension_01-Jun-2024_15:53:37.csv')
survey_table = pd.read_csv('../csv_from_excel/survey_table.csv')



In [27]:
national_results.dtypes

seriesID       object
year            int64
period         object
period_name    object
value          object
footnotes      object
dtype: object

In [43]:
national_results['value'] = national_results['value'].replace('-', 0)
national_results['value'] = national_results['value'].astype('float')

national_results.dtypes

seriesID        object
year             int64
period          object
period_name     object
value          float64
footnotes       object
dtype: object

In [44]:
state_results['value'] = state_results['value'].replace('-', 0)
state_results['value'] = state_results['value'].astype('float')

state_results.dtypes

seriesID        object
year             int64
period          object
period_name     object
value          float64
footnotes       object
dtype: object

In [45]:

def space_remover(df):

        for col in df.columns:

                df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
                
        return df


national_results = space_remover(national_results)
national_series_dimension = space_remover(national_series_dimension)
state_results = space_remover(state_results)
state_series_dimension = space_remover(state_series_dimension)
survey_table = space_remover(survey_table)
                

In [46]:
pattern1 = re.compile(', seasonally adjusted')
pattern2 = re.compile(', not seasonally adjusted')

national_series_dimension['is_adjusted'] = \
    national_series_dimension['series'].str.lower().apply(lambda x: True if re.search(pattern1, x) and not re.search(pattern2, x) else False if re.search(pattern2, x) else None)

state_series_dimension['is_adjusted'] = \
    state_series_dimension['series'].str.lower().apply(lambda x: True if re.search(pattern1, x) and not re.search(pattern2, x) else False if re.search(pattern2, x) else None)


In [48]:
# national_series_dimension.series = national_series_dimension['series'].str.lower().replace('seasonally adjusted', '').replace('not', '')
# state_series_dimension['series'] = state_series_dimension['series'].str.replace('Seasonally adjusted', '').replace('Not', '')

def adjusted_remover(series):
    term1 = ', seasonally adjusted'
    term2 = ', not seasonally adjusted'
    
    def remove_terms(text):
        text_lower = text.lower()
        index1 = text_lower.find(term1)
        index2 = text_lower.find(term2)
        if index1 != -1:
            return text[:index1]
        elif index2 != -1:
            return text[:index2]
        return text
    
    # Apply the removal function to the 'series' column
    series['series'] = series['series'].apply(remove_terms)
    return series

# adjusted_remover(national_series_dimension)
national_series_dimension = adjusted_remover(national_series_dimension)
state_series_dimension = adjusted_remover(state_series_dimension)



In [51]:
national_series_dimension[(national_series_dimension['series'].str.contains('seasonally adjusted') == True)].sum(),\
state_series_dimension[(state_series_dimension['series'].str.contains('seasonally adjusted') == True)].sum()

(series         0
 seriesID       0
 survey         0
 is_adjusted    0
 dtype: object,
 series         0
 seriesID       0
 state          0
 survey         0
 is_adjusted    0
 dtype: object)

In [52]:
national_series_dimension.is_adjusted.value_counts()

is_adjusted
True     41
False    11
Name: count, dtype: int64

In [54]:
state_series_dimension.is_adjusted.value_counts()

is_adjusted
False    683
True     666
Name: count, dtype: int64

In [55]:
national_series_dimension.to_csv('../cleaning/output/national_seriesID_list_cleaned.csv', index=False)
state_series_dimension.to_csv('../cleaning/output/state_series_dimension.csv_cleaned.csv', index=False)